In [1]:
import os
from tqdm import tqdm
import boto3

os.environ['AWS_PROFILE'] = 'burner-llm'

In [2]:
cf_client = boto3.client('cloudformation')
stackname = 'LLMStackS3'

response = cf_client.describe_stacks(StackName=stackname)
outputs = response["Stacks"][0]["Outputs"]

In [3]:
cf_outputs = {}
for i in outputs:
    cf_outputs[i['OutputKey']] = i['OutputValue']

In [4]:
cf_outputs

{'DataUploaderRoleArn': 'arn:aws:iam::878691224389:role/data-uploader-role',
 'DataUploaderRoleName': 'data-uploader-role',
 'S3BucketName': 'llmstacks3-s3bucket-1b39zw19b74me'}

In [7]:
BUCKET_NAME = cf_outputs['S3BucketName']
DATA_UPLOADER_ROLE_ARN = cf_outputs['DataUploaderRoleArn']

DATA_PATH = 'data/sagemaker_documentation/'

In [8]:
sts_client = boto3.client('sts')

assumed_role_object=sts_client.assume_role(
    RoleArn=DATA_UPLOADER_ROLE_ARN,
    RoleSessionName="AssumeRoleSession1"
)

credentials=assumed_role_object['Credentials']

s3_resource=boto3.resource(
    's3',
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
)


In [10]:
file_names_list = os.listdir(DATA_PATH)
file_paths_list = [os.path.join(DATA_PATH, i) for i in file_names_list]

print(f'There are {len(file_paths_list)} files.')

There are 336 files.


In [11]:
for filename, filepath in tqdm(zip(file_names_list, file_paths_list)):    
    s3_resource.Bucket(BUCKET_NAME).upload_file(filepath, os.path.join('input_data', filename))

336it [01:40,  3.35it/s]


In [ ]:
# NOTE: Uploading for sure can be done in more efficient way.
# Either bulk update or zipping files locally, sending the package and unzipping directly on S3.